## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
from keras.applications import *
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
#model=Xception(input_shape=(32, 32, 3), weights='imagenet', include_top=False)
'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [8]:
x = model.output

'''可以參考Cifar10實作章節'''
x = Flatten()(x)
x = Dropout(0.5)(x)
predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 178


e:\Using\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [9]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


e:\Using\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [11]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 204s 4ms/step - loss: 1.7843 - accuracy: 0.5340
Epoch 2/100
50000/50000 [==============================] - 197s 4ms/step - loss: 1.4364 - accuracy: 0.6204
Epoch 3/100
50000/50000 [==============================] - 195s 4ms/step - loss: 1.1867 - accuracy: 0.6573
Epoch 4/100
50000/50000 [==============================] - 196s 4ms/step - loss: 1.0329 - accuracy: 0.6920
Epoch 5/100
50000/50000 [==============================] - 196s 4ms/step - loss: 0.8854 - accuracy: 0.7239
Epoch 6/100
50000/50000 [==============================] - 198s 4ms/step - loss: 0.8120 - accuracy: 0.7436
Epoch 7/100
50000/50000 [==============================] - 194s 4ms/step - loss: 0.7166 - accuracy: 0.7672
Epoch 8/100
50000/50000 [==============================] - 197s 4ms/step - loss: 0.6304 - accuracy: 0.7962
Epoch 9/100
50000/50000 [==============================] - 198s 4ms/step - loss: 0.5771 - accuracy: 0.8135
Epoch 10/100
50000/50000 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



50000/50000 [==============================] - 199s 4ms/step - loss: 0.1868 - accuracy: 0.9417
Epoch 24/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.1845 - accuracy: 0.9445
Epoch 25/100
50000/50000 [==============================] - 199s 4ms/step - loss: 0.1626 - accuracy: 0.9496
Epoch 26/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.1513 - accuracy: 0.9515
Epoch 27/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.2053 - accuracy: 0.9423
Epoch 28/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.1609 - accuracy: 0.9526
Epoch 29/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.1593 - accuracy: 0.9537
Epoch 30/100
50000/50000 [==============================] - 199s 4ms/step - loss: 0.1480 - accuracy: 0.9556
Epoch 31/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.1474 - accuracy: 0.9557
Epoch 32/100
50000/50000 [===============

Epoch 99/100
50000/50000 [==============================] - 193s 4ms/step - loss: 0.0508 - accuracy: 0.9868
Epoch 100/100
50000/50000 [==============================] - 193s 4ms/step - loss: 0.0566 - accuracy: 0.9868
